In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
df=pd.read_csv(r'pcs_ARASAT.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.set_index('Timestamp')
df.head()

,acAR,acAS,acAT,acPowerFactor,actKw
Timestamp,,,,,
2022-04-19 11:32:00,89.3,89.9,87.6,9.98,-62.8
2022-04-19 11:32:00,89.3,89.9,87.6,9.98,-62.8
2022-04-19 11:32:00,89.5,89.7,88.0,9.98,-62.8
2022-04-19 11:32:00,89.5,89.7,88.0,9.98,-62.8
2022-04-19 11:32:00,87.8,88.3,86.8,9.98,-61.7


In [4]:
daily_df = df.copy()
print(len(daily_df))
daily_df.head()


8292


,acAR,acAS,acAT,acPowerFactor,actKw
Timestamp,,,,,
2022-04-19 11:32:00,89.3,89.9,87.6,9.98,-62.8
2022-04-19 11:32:00,89.3,89.9,87.6,9.98,-62.8
2022-04-19 11:32:00,89.5,89.7,88.0,9.98,-62.8
2022-04-19 11:32:00,89.5,89.7,88.0,9.98,-62.8
2022-04-19 11:32:00,87.8,88.3,86.8,9.98,-61.7


In [5]:
train_df,test_df = daily_df[:7500], daily_df[7500:] 


In [6]:
print((train_df).shape)

(7500, 5)


In [8]:
from preprocessing_ARASAT import fit_traindata,trans_data
scalers,train = fit_traindata(train_df)
test = trans_data(test_df,scalers)

Index(['acAR', 'acAS', 'acAT', 'acPowerFactor', 'actKw'], dtype='object')


C:\Users\Hoang\OneDrive - 국민대학교\HOANG (Dont Open)\2021==AI Lab Job\11.Exhibition\preprocessing_ARASAT.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
C:\Users\Hoang\OneDrive - 국민대학교\HOANG (Dont Open)\2021==AI Lab Job\11.Exhibition\preprocessing_ARASAT.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
C:\Users\Hoang\OneDrive - 국민대학교\HOANG (Dont Open)\2021==AI Lab Job\11.Exhibition\preprocessing_ARASAT.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [9]:
def split_series(series, n_past, n_future):
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

In [10]:
n_past = 120
n_future = 12 
n_features = 5

In [11]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [12]:
print(X_train)

[[[0.75587703 0.76079137 0.75448029 0.91666667 0.23273657]
  [0.75587703 0.76079137 0.75448029 0.91666667 0.23273657]
  [0.75949367 0.75719424 0.76164875 0.91666667 0.23273657]
  ...
  [0.80650995 0.8057554  0.80824373 0.91666667 0.18158568]
  [0.80650995 0.80935252 0.80824373 0.91666667 0.18158568]
  [0.80650995 0.80935252 0.80824373 0.91666667 0.18158568]]

 [[0.75587703 0.76079137 0.75448029 0.91666667 0.23273657]
  [0.75949367 0.75719424 0.76164875 0.91666667 0.23273657]
  [0.75949367 0.75719424 0.76164875 0.91666667 0.23273657]
  ...
  [0.80650995 0.80935252 0.80824373 0.91666667 0.18158568]
  [0.80650995 0.80935252 0.80824373 0.91666667 0.18158568]
  [0.81374322 0.8057554  0.80645161 0.91666667 0.18158568]]

 [[0.75949367 0.75719424 0.76164875 0.91666667 0.23273657]
  [0.75949367 0.75719424 0.76164875 0.91666667 0.23273657]
  [0.72875226 0.73201439 0.74014337 0.91666667 0.26086957]
  ...
  [0.80650995 0.80935252 0.80824373 0.91666667 0.18158568]
  [0.81374322 0.8057554  0.8064516

In [13]:
print(y_train)

[[[0.81374322 0.8057554  0.80645161 0.91666667 0.18158568]
  [0.81374322 0.8057554  0.80645161 0.91666667 0.18158568]
  [0.80831826 0.81115108 0.81362007 0.91666667 0.18158568]
  ...
  [0.83905967 0.8381295  0.83512545 0.91666667 0.15345269]
  [0.83544304 0.83273381 0.83512545 1.         0.15601023]
  [0.83544304 0.83273381 0.83512545 1.         0.15601023]]

 [[0.81374322 0.8057554  0.80645161 0.91666667 0.18158568]
  [0.80831826 0.81115108 0.81362007 0.91666667 0.18158568]
  [0.80831826 0.81115108 0.81362007 0.91666667 0.18158568]
  ...
  [0.83544304 0.83273381 0.83512545 1.         0.15601023]
  [0.83544304 0.83273381 0.83512545 1.         0.15601023]
  [0.8119349  0.81115108 0.80645161 0.91666667 0.18158568]]

 [[0.80831826 0.81115108 0.81362007 0.91666667 0.18158568]
  [0.80831826 0.81115108 0.81362007 0.91666667 0.18158568]
  [0.80831826 0.80935252 0.80824373 1.         0.18158568]
  ...
  [0.83544304 0.83273381 0.83512545 1.         0.15601023]
  [0.8119349  0.81115108 0.8064516

In [14]:
print(X_train.shape,y_train.shape)

(7369, 120, 5) (7369, 12, 5)


In [15]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 5)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100), (None, 42400       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 12, 100)      0           lstm[0][0]                       
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 12, 100)      80400       repeat_vector[0][0]              
                                                                 lstm[0][1]                   

In [16]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)


In [17]:
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),batch_size=1024,verbose=1,callbacks=[reduce_lr])


Epoch 1/50
8/8 [==============================] - 9s 166ms/step - loss: 0.1775 - val_loss: 0.0234
Epoch 2/50
8/8 [==============================] - 0s 26ms/step - loss: 0.0163 - val_loss: 0.0092
Epoch 3/50
8/8 [==============================] - 0s 25ms/step - loss: 0.0079 - val_loss: 0.0028
Epoch 4/50
8/8 [==============================] - 0s 24ms/step - loss: 0.0045 - val_loss: 0.0035
Epoch 5/50
8/8 [==============================] - 0s 24ms/step - loss: 0.0035 - val_loss: 8.7217e-04
Epoch 6/50
8/8 [==============================] - 0s 23ms/step - loss: 0.0024 - val_loss: 0.0010
Epoch 7/50
8/8 [==============================] - 0s 24ms/step - loss: 0.0019 - val_loss: 7.0112e-04
Epoch 8/50
8/8 [==============================] - 0s 23ms/step - loss: 0.0014 - val_loss: 5.9382e-04
Epoch 9/50
8/8 [==============================] - 0s 24ms/step - loss: 0.0012 - val_loss: 4.3240e-04
Epoch 10/50
8/8 [==============================] - 0s 24ms/step - loss: 0.0011 - val_loss: 4.0533e-04
Epoch 11

In [18]:
pred_e1d = model_e1d1.predict(X_train)

In [19]:
pred_e1d1=model_e1d1.predict(X_test)


In [20]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    pred_e1d[:,:,index]=scaler.inverse_transform(pred_e1d[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [21]:
from sklearn.metrics import mean_absolute_error
for index,i in enumerate(train_df.columns):
    print(i)
    for j in range(1,12):
        print("Second ",j,":")
        print("MAE-E1D1 : ",mean_absolute_error(y_test[:,j-1,index],pred_e1d1[:,j-1,index]),end=", ")
        print()
        print()

acAR
Second  1 :
MAE-E1D1 :  1.5090180320566613, 

Second  2 :
MAE-E1D1 :  0.6494379202284356, 

Second  3 :
MAE-E1D1 :  0.5453142474891552, 

Second  4 :
MAE-E1D1 :  0.5794753820560697, 

Second  5 :
MAE-E1D1 :  0.6480761483289108, 

Second  6 :
MAE-E1D1 :  0.737303106218531, 

Second  7 :
MAE-E1D1 :  0.8196122508546766, 

Second  8 :
MAE-E1D1 :  0.88069400758498, 

Second  9 :
MAE-E1D1 :  0.920327107808954, 

Second  10 :
MAE-E1D1 :  0.9425664258255717, 

Second  11 :
MAE-E1D1 :  0.9525088404022076, 

acAS
Second  1 :
MAE-E1D1 :  2.0501824521802012, 

Second  2 :
MAE-E1D1 :  0.715773310351119, 

Second  3 :
MAE-E1D1 :  0.6461652954840263, 

Second  4 :
MAE-E1D1 :  0.6784825242774031, 

Second  5 :
MAE-E1D1 :  0.694635942375425, 

Second  6 :
MAE-E1D1 :  0.7555009596647183, 

Second  7 :
MAE-E1D1 :  0.845208073095186, 

Second  8 :
MAE-E1D1 :  0.9248528831124123, 

Second  9 :
MAE-E1D1 :  0.9836752425524179, 

Second  10 :
MAE-E1D1 :  1.0200657716136474, 

Second  11 :
MAE-E1D1 :  1.0

In [22]:
print(pred_e1d[3])

[[ 91.526855   91.43475    89.48658     9.984192  -65.159775 ]
 [ 92.28411    92.80291    90.75154     9.985487  -64.89897  ]
 [ 92.48851    93.222946   91.18429     9.985963  -64.73582  ]
 [ 92.46702    93.18628    91.196335    9.9858055 -64.666565 ]
 [ 92.38593    92.99736    91.04603     9.985511  -64.65298  ]
 [ 92.32489    92.803444   90.86946     9.98528   -64.67081  ]
 [ 92.2857     92.637314   90.70441     9.985179  -64.69671  ]
 [ 92.278984   92.52198    90.56452     9.9852085 -64.73426  ]
 [ 92.29025    92.45174    90.45697     9.985337  -64.7729   ]
 [ 92.32391    92.42447    90.37747     9.985533  -64.80796  ]
 [ 92.35674    92.41091    90.31832     9.9857645 -64.842606 ]
 [ 92.40367    92.42514    90.272415    9.98604   -64.87181  ]]


In [23]:
print(y_train[0])

[[ 92.5   92.4   90.5    9.98 -64.8 ]
 [ 92.5   92.4   90.5    9.98 -64.8 ]
 [ 92.2   92.7   90.9    9.98 -64.8 ]
 [ 92.2   92.7   90.9    9.98 -64.8 ]
 [ 92.2   92.6   90.6    9.99 -64.8 ]
 [ 92.2   92.6   90.6    9.99 -64.8 ]
 [ 92.1   92.8   90.5    9.98 -64.8 ]
 [ 92.1   92.8   90.5    9.98 -64.8 ]
 [ 93.9   94.2   92.1    9.98 -65.9 ]
 [ 93.9   94.2   92.1    9.98 -65.9 ]
 [ 93.7   93.9   92.1    9.99 -65.8 ]
 [ 93.7   93.9   92.1    9.99 -65.8 ]]


In [24]:
model_e1d1.save('mmLSTM_ARASAT.h5')